# RAG-powered Text-To-SQL using PhiData, Ollama and Argilla

In this recipe, we will show how to create a powerful text-to-SQL engine that is powered by a RAG pipeline with [PhiData](https://docs.phidata.com/). We will use the [Ollama](https://github.com/ollama/ollama) to run the LLM and [DuckDB](https://duckdb.org/) powered to [run the SQL on top of datasets on the Hugging Face Hub](https://duckdb.org/2024/05/29/access-150k-plus-datasets-from-hugging-face-with-duckdb.html). Eventually, we will use Argilla to structure the data and review the textual prompts. 

## Getting started

### Deploy the Argilla server¶

If you already have deployed Argilla, you can skip this step. Otherwise, you can quickly deploy Argilla following [this guide](https://docs.argilla.io/latest/getting_started/quickstart/).

### Install dependencies

Since we will be using Ollama, we recommend installing ollama as [described on their GitHub repo](https://github.com/ollama/ollama). After that, we can deploy our ollama server and download and serve required model with.

```bash
ollama serve
```

```bash
ollama run llama3.2
```

Besides that, we will also be needing some Python dependencies. You can install them by running:

In [12]:
!pip install -qqq bs4 phidata lancedb duckdb ollama datasets argilla 

## Create our RAG pipeline

We want our model to have a better understanding of the DuckDB documentation and information. We will therefore

In [11]:
from phi.knowledge.website import WebsiteKnowledgeBase
from phi.vectordb.lancedb import LanceDb
from phi.embedder.ollama import OllamaEmbedder
from phi.document.reader.website import WebsiteReader

knowledge_base = WebsiteKnowledgeBase(
    reader=WebsiteReader(chunk=False),
    urls=[
        "https://duckdb.org/docs/sql/statements/overview", 
        "https://duckdb.org/docs/sql/data_types/overview",
        "https://duckdb.org/docs/sql/expressions/overview",
        "https://duckdb.org/docs/sql/functions/overview",
        "https://duckdb.org/docs/sql/dialect/overview"
    ],
    max_links=1000,
    vector_db=LanceDb(
        embedder=OllamaEmbedder(model="llama3.1")
    ),
    max_depth=2
)
knowledge_base.load(recreate=True)

INFO     Creating table: phi                                                    
INFO     Creating table: phi                                                    
INFO     Loading knowledge base                                                 


[2024-10-08T14:07:21Z WARN  lance::dataset] No existing dataset at /tmp/lancedb/phi.lance, it will be created


INFO     Loaded 10 documents to knowledge base                                  
INFO     Loaded 20 documents to knowledge base                                  
INFO     Loaded 30 documents to knowledge base                                  
INFO     Loaded 40 documents to knowledge base                                  
INFO     Loaded 50 documents to knowledge base                                  


## Get dataset information

Hugging Face datasets are hosted publicly but in order to generate synthetic text queries and their corresponding SQL, we will first need to obtain some informationf about this datasets. We will use this with the Hugging Face Hub API.

### List Hugging Face datasets

In [70]:
from huggingface_hub import HfApi

datasets = list(HfApi().list_datasets(limit=100, gated=False, expand=["description"]))
datasets[0]

DatasetInfo(id='google/frames-benchmark', author=None, sha=None, created_at=None, last_modified=None, private=None, gated=None, disabled=None, downloads=None, downloads_all_time=None, likes=None, paperswithcode_id=None, tags=None, trending_score=52, card_data=None, siblings=None)

### Get dataset descriptions

In [155]:
from huggingface_hub import ModelCard

card_content = []
for dataset in datasets:
    try:
        card_content.append("---".join(ModelCard.load(dataset.id, repo_type="dataset").content.split("---")[2:]).strip())
    except Exception:
        if hasattr(dataset, "description"):
            card_content.append(dataset.description)
        else:
            card_content.append(None)
card_content[0]

Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.
Repo card metadata block was not found. Setting CardData to empty.


'# FRAMES: Factuality, Retrieval, And reasoning MEasurement Set\n\nFRAMES is a comprehensive evaluation dataset designed to test the capabilities of Retrieval-Augmented Generation (RAG) systems across factuality, retrieval accuracy, and reasoning.\nOur paper with details and experiments is available on arXiv: [https://arxiv.org/abs/2409.12941](https://arxiv.org/abs/2409.12941).\n\n\n## Dataset Overview\n\n- 824 challenging multi-hop questions requiring information from 2-15 Wikipedia articles\n- Questions span diverse topics including history, sports, science, animals, health, etc.\n- Each question is labeled with reasoning types: numerical, tabular, multiple constraints, temporal, and post-processing\n- Gold answers and relevant Wikipedia articles provided for each question\n\n## Key Features\n\n- Tests end-to-end RAG capabilities in a unified framework\n- Requires integration of information from multiple sources\n- Incorporates complex reasoning and temporal disambiguation\n- Designe

### Get dataset info from the datasets-server

In [173]:
import requests

def get_dataset_info(dataset_id):
    url = f"https://datasets-server.huggingface.co/info?dataset={dataset_id}"
    response = requests.get(url)
    data = response.json()
    if data.get("dataset_info"):
        dataset_info = data["dataset_info"]
        dataset_config = list(dataset_info.keys())[0]
        dataset_split = list(dataset_info[dataset_config]["splits"].keys())[0]
        dataset_size = dataset_info[dataset_config]["splits"][dataset_split]["num_examples"]
        dataset_features = dataset_info[dataset_config]["features"]
        return dataset_id, dataset_config, dataset_split, dataset_size, dataset_features
    else:
        return None, None, None, None

dataset_info = [get_dataset_info(dataset.id) for dataset in datasets]
dataset_info[0]

('google/frames-benchmark',
 'default',
 'test',
 824,
 {'Unnamed: 0': {'dtype': 'int64', '_type': 'Value'},
  'Prompt': {'dtype': 'string', '_type': 'Value'},
  'Answer': {'dtype': 'string', '_type': 'Value'},
  'wikipedia_link_1': {'dtype': 'string', '_type': 'Value'},
  'wikipedia_link_2': {'dtype': 'string', '_type': 'Value'},
  'wikipedia_link_3': {'dtype': 'string', '_type': 'Value'},
  'wikipedia_link_4': {'dtype': 'string', '_type': 'Value'},
  'wikipedia_link_5': {'dtype': 'string', '_type': 'Value'},
  'wikipedia_link_6': {'dtype': 'string', '_type': 'Value'},
  'wikipedia_link_7': {'dtype': 'string', '_type': 'Value'},
  'wikipedia_link_8': {'dtype': 'string', '_type': 'Value'},
  'wikipedia_link_9': {'dtype': 'string', '_type': 'Value'},
  'wikipedia_link_10': {'dtype': 'string', '_type': 'Value'},
  'wikipedia_link_11+': {'dtype': 'string', '_type': 'Value'},
  'reasoning_types': {'dtype': 'string', '_type': 'Value'},
  'wiki_links': {'dtype': 'string', '_type': 'Value'}})

### Get first row of data data from the datasets-server




In [174]:
import random

def get_dataset_data(dataset_id, config, split, offset=0, length=1):
    if dataset_id and config and split:
        url = f"https://datasets-server.huggingface.co/rows?dataset={dataset_id}&config={config}&split={split}&offset={offset}&length={length}"
        response = requests.get(url)
        data = response.json()
        if data.get("rows"):
            return [row["row"] for row in data["rows"]]
        else:
            return None
    else:
        return None

dataset_data = [get_dataset_data(dataset.id, config, split, random.choice(list(range(size)))) for dataset, (id, config, split, size, features) in zip(datasets, dataset_info)]
dataset_data[0]

ValueError: not enough values to unpack (expected 5, got 4)

## Synthesize data

### Construct synthetic SQL Schemas

In [68]:
from phi.assistant import Assistant
from phi.llm.ollama import Ollama

prompt_table_schema = """Write a SQL table schema, given a JSON schema and a row of example data. Only return the schema as plain text and nothing else.

Schema:
{features}

Example data:
{data}
"""

def get_table_schema(features, data):
    if features and data:
        assistant = Assistant(
            llm=Ollama(model="llama3.1"),
            show_tool_calls=False,
            knowledge_base=knowledge_base
        )
        return assistant.run(prompt_table_schema.format(features=features, data=data), stream=False)
    else:
        return None

# sql_schemas: list[Iterator[str] | str | BaseModel | None] = [get_table_schema(features[-1], data) for features, data in zip(dataset_info, dataset_data)]
table_schema = get_table_schema(features=dataset_info[0][-1], data=dataset_data[0])

### Construct synthetic natural language queries 

In [172]:
nsql_query_prompt = """Come up with a reasonable, short and concise natural language query that can be answered using a SQL query on top of the table. Also consider basic aggregates and other operations. Do this using a table schema, a table description, and example data, but consider this is only a single row in the dataset. Only return the natural language query as plain text and nothing else.

Description:
{description}

Schema:
{schema}

Example data:
{data}
"""

def get_nsql_query(schema, description, data):
    assistant = Assistant(
        llm=Ollama(model="llama3.1"),
        show_tool_calls=False,
        knowledge_base=knowledge_base
    )
    return assistant.run(nsql_query_prompt.format(schema=schema, description=description, data=data[1]), stream=False)
    
nsql = get_nsql_query(schema=table_schema, description=card_content[0], data=dataset_data[0])
nsql

'What is the height of Bronte Tower in feet, and what rank would it hold among the tallest buildings in New York City as of August 2024?'

### Construct synthetic SQL queries

In [166]:
card_content[0]

'# FRAMES: Factuality, Retrieval, And reasoning MEasurement Set\n\nFRAMES is a comprehensive evaluation dataset designed to test the capabilities of Retrieval-Augmented Generation (RAG) systems across factuality, retrieval accuracy, and reasoning.\nOur paper with details and experiments is available on arXiv: [https://arxiv.org/abs/2409.12941](https://arxiv.org/abs/2409.12941).\n\n\n## Dataset Overview\n\n- 824 challenging multi-hop questions requiring information from 2-15 Wikipedia articles\n- Questions span diverse topics including history, sports, science, animals, health, etc.\n- Each question is labeled with reasoning types: numerical, tabular, multiple constraints, temporal, and post-processing\n- Gold answers and relevant Wikipedia articles provided for each question\n\n## Key Features\n\n- Tests end-to-end RAG capabilities in a unified framework\n- Requires integration of information from multiple sources\n- Incorporates complex reasoning and temporal disambiguation\n- Designe

In [169]:
nsql_to_sql_prompt = """Write a valid and fully complete DuckDB SQL query that would be able to provide a good answer to a natural language query, given a dataset scription, table schema and example data. Think step by step and make sure to include all relevant columns and potential statements. Only return the SQL query as plain text and nothing else.

Description:
{description}

Schema:
{schema}

Natural language query:
{nsql}
"""

def get_nsql_to_sql(description, schema, data, nsql):
    assistant = Assistant(
        llm=Ollama(model="llama3.1"),
        show_tool_calls=False,
        knowledge_base=knowledge_base,
        system_prompt="You are a SQL expert that understand business scenarios and complexities of natural language question."
    )
    return assistant.run(nsql_to_sql_prompt.format(schema=schema, description=description, data=data[0], nsql=nsql), stream=False)

get_nsql_to_sql(schema=table_schema, description=card_content[0], data=dataset_data[0], nsql=nsql)    

"SELECT T1.wikipedia_link_3 \nFROM table_name AS T1 \nJOIN table_name AS T2 ON T1.Unnamed_0 = T2.Unnamed_0 \nWHERE T2.Prompt LIKE '%Dewey Decimal Classification%' AND T2.Answer LIKE '%Jane Eyre%' AND T2.reasoning_types LIKE 'numerical';"

In [95]:
dataset_data[0]

{'row_idx': 0,
 'row': {'Unnamed: 0': 0,
  'Prompt': "If my future wife has the same first name as the 15th first lady of the United States' mother and her surname is the same as the second assassinated president's mother's maiden name, what is my future wife's name? ",
  'Answer': 'Jane Ballou',
  'wikipedia_link_1': 'https://en.wikipedia.org/wiki/President_of_the_United_States',
  'wikipedia_link_2': 'https://en.wikipedia.org/wiki/James_Buchanan',
  'wikipedia_link_3': 'https://en.wikipedia.org/wiki/Harriet_Lane',
  'wikipedia_link_4': 'https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States_who_died_in_office',
  'wikipedia_link_5': 'https://en.wikipedia.org/wiki/James_A._Garfield',
  'wikipedia_link_6': None,
  'wikipedia_link_7': None,
  'wikipedia_link_8': None,
  'wikipedia_link_9': None,
  'wikipedia_link_10': None,
  'wikipedia_link_11+': None,
  'reasoning_types': 'Multiple constraints',
  'wiki_links': "['https://en.wikipedia.org/wiki/President_of_the_United_St

## Validate data Argilla

### Create Argilla dataset

In [ ]:
import argilla as rg

settings = rg.Settings(
    fields=[
        rg.TextField(name="nlq", title="Natural Language Query"),
        rg.TextField(name="sql", title="SQL Query", use_markdown=True),
        rg.TextField(name="dataset_viewer", title="Dataset viewer used to validate the query.", use_markdown=True),
        rg.TextField(name="schema", title="Database schema", use_markdown=True),
        rg.TextField(name="description", title="Dataset description", use_markdown=True),
    ],
    questions=[
        rg.TextQuestion(name="nlq", title="Natural Language Query", description="Check if the query makes sense and correct it otherwise."),
        rg.TextQuestion(name="sql", title="SQL Query", description="Check if the SQL query is valid and correct if not use the dataset viewer to iterated and paste the improved SQL query here.", use_markdown=True)
    ]
)

dataset = rg.Dataset(name="text-to-sql", workspace="argilla", settings=settings)
dataset.create()

### Upload data to argilla

In [ ]:
import urllib

def get_dataset_viewer(dataset_id, sql_query):
    sql_query = urllib.parse.quote(sql_query)
    url = f"https://huggingface.co/datasets/{dataset_id}/embed/viewer?sql_console=true&sql={sql_query}"
    iframe = f"""
    <iframe
  src="{url}"
  frameborder="0"
  width="100%"
  height="800px"
></iframe>
"""
    return iframe

In [ ]:
records = [
    rg.Record(
        fields={
            "schema": "",
            "nlq": "",
            "sql": "",
            "dataset_viewer": ""
        },
        question={
            "nlq": "",
            "sql": ""
        }
    ) for schema, 
]